# Clases desbalanceadas

Conjunto de datos de entrenamiento con clases minoritarias, por lo que la información esta sesagada

## Podemos resolver el problema de diferentes formas: 

**Ajuste de Parámetros del modelo:** Ajustar la métrica del modelo para equilibrar a la clase minoritaria, dando un peso diferente durante el entrenamiento.

**Modificar el Dataset:**  Eliminar datos de la clase mayoritaria para reducirla.

**Muestras artificiales:** Crear muestras sintéticas utilizando algoritmos que intentan seguir la tendencia del grupo minoritario. 

**Ensamble de métodos:** Entrenar diversos modelos y entre todos obtener el resultado final.


https://www.aprendemachinelearning.com/clasificacion-con-datos-desbalanceados/

### Biblioteca: imbalanced-learn

https://pypi.org/project/imbalanced-learn/

$ pip install imbalanced-learn

# Credit Card Fraud Detection

https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud?select=creditcard.csv

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns 
import pandas as pd
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression # Importamos la clase de Regresión Lineal de scikit-learn
from sklearn.metrics import mean_squared_error, r2_score # error
from sklearn.metrics import classification_report
# conda install -c conda-forge imbalanced-learn
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek


from sklearn import metrics
from collections import Counter  

In [2]:
mainpath = "./datasets/"
filename = 'creditcard.csv'
fullpath = os.path.join(mainpath, filename)
dataset= pd.read_csv (fullpath)


In [ ]:
# mostrar información del DataFrame
def info(df):
    display(df.head(10))
    print()
    print(df.info())
    print()
    print(df.describe())
    print()
    print('Duplicated: ',df.duplicated().sum())
    print()
    print('Null values %:')
    print(100*df.isnull().sum()/len(df))

info(dataset)

In [4]:
#eleminar duplicados

dataset.drop_duplicates(inplace=True)


In [ ]:
# contar valores de la columna Class
print(dataset['Class'].value_counts())


<div class="alert alert-block alert-info">
<b>

* 284807 entradas y 31 características 30 float y la categoría a predecir int
* 1081 duplicados eliminados
* No hay valores ausentes
* Clase desbalanceadas: 

        0    283253
        1       473
</b></div>

### Seleccionar características y separación en conjunto de entrenamiento y conjunto de prueba

In [6]:
features = dataset.drop(columns=['Class'])
target = dataset['Class']
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.25, random_state=42)

### Evaluar modelo de regresión logística

In [7]:
def run_model(X_train, X_test, Y_train, Y_test):
    clf_base = LogisticRegression(C=1.0,penalty='l2',random_state=1,solver="newton-cg")
    clf_base.fit(X_train, Y_train)
    return clf_base

In [8]:

def evaluate_model(model, train_features, train_target, test_features, test_target):
   
    eval_stats = {}
    
    fig, axs = plt.subplots(1, 3, figsize=(20, 6)) 
    
    for type, features, target in (('train', train_features, train_target), ('test', test_features, test_target)):
        
        eval_stats[type] = {}
    
        pred_target = model.predict(features)
        pred_proba = model.predict_proba(features)[:, 1]
        
        # F1
        f1_thresholds = np.arange(0, 1.01, 0.05)
        f1_scores = [metrics.f1_score(target, pred_proba>=threshold) for threshold in f1_thresholds]
        
        # ROC
        fpr, tpr, roc_thresholds = metrics.roc_curve(target, pred_proba)
        roc_auc = metrics.roc_auc_score(target, pred_proba)    
        eval_stats[type]['ROC AUC'] = roc_auc

        # PRC
        precision, recall, pr_thresholds = metrics.precision_recall_curve(target, pred_proba)
        aps = metrics.average_precision_score(target, pred_proba)
        eval_stats[type]['APS'] = aps
        
        if type == 'train':
            color = 'blue'
        else:
            color = 'green'

        # Valor F1
        ax = axs[0]
        max_f1_score_idx = np.argmax(f1_scores)
        ax.plot(f1_thresholds, f1_scores, color=color, label=f'{type}, max={f1_scores[max_f1_score_idx]:.2f} @ {f1_thresholds[max_f1_score_idx]:.2f}')
        # establecer cruces para algunos umbrales        
        for threshold in (0.2, 0.4, 0.5, 0.6, 0.8):
            closest_value_idx = np.argmin(np.abs(f1_thresholds-threshold))
            marker_color = 'orange' if threshold != 0.5 else 'red'
            ax.plot(f1_thresholds[closest_value_idx], f1_scores[closest_value_idx], color=marker_color, marker='X', markersize=7)
        ax.set_xlim([-0.02, 1.02])    
        ax.set_ylim([-0.02, 1.02])
        ax.set_xlabel('threshold')
        ax.set_ylabel('F1')
        ax.legend(loc='lower center')
        ax.set_title(f'Valor F1') 

        # ROC
        ax = axs[1]    
        ax.plot(fpr, tpr, color=color, label=f'{type}, ROC AUC={roc_auc:.2f}')
        # establecer cruces para algunos umbrales        
        for threshold in (0.2, 0.4, 0.5, 0.6, 0.8):
            closest_value_idx = np.argmin(np.abs(roc_thresholds-threshold))
            marker_color = 'orange' if threshold != 0.5 else 'red'            
            ax.plot(fpr[closest_value_idx], tpr[closest_value_idx], color=marker_color, marker='X', markersize=7)
        ax.plot([0, 1], [0, 1], color='grey', linestyle='--')
        ax.set_xlim([-0.02, 1.02])    
        ax.set_ylim([-0.02, 1.02])
        ax.set_xlabel('FPR')
        ax.set_ylabel('TPR')
        ax.legend(loc='lower center')        
        ax.set_title(f'Curva ROC')
        
        # PRC
        ax = axs[2]
        ax.plot(recall, precision, color=color, label=f'{type}, AP={aps:.2f}')
        # establecer cruces para algunos umbrales        
        for threshold in (0.2, 0.4, 0.5, 0.6, 0.8):
            closest_value_idx = np.argmin(np.abs(pr_thresholds-threshold))
            marker_color = 'orange' if threshold != 0.5 else 'red'
            ax.plot(recall[closest_value_idx], precision[closest_value_idx], color=marker_color, marker='X', markersize=7)
        ax.set_xlim([-0.02, 1.02])    
        ax.set_ylim([-0.02, 1.02])
        ax.set_xlabel('recall')
        ax.set_ylabel('precision')
        ax.legend(loc='lower center')
        ax.set_title(f'PRC')   
        
        eval_stats[type]['Exactitud'] = metrics.accuracy_score(target, pred_target)
        eval_stats[type]['F1'] = metrics.f1_score(target, pred_target)
    
    df_eval_stats = pd.DataFrame(eval_stats)
    df_eval_stats = df_eval_stats.round(2)
    df_eval_stats = df_eval_stats.reindex(index=('Exactitud', 'F1', 'APS', 'ROC AUC'))
    
    print(df_eval_stats)
    
    return eval_stats['train']['F1'], eval_stats['test']['F1']


In [ ]:

model = run_model(X_train, X_test, y_train, y_test)
train_f1, test_f1 = evaluate_model(model, X_train, y_train, X_test, y_test)

In [ ]:
#matriz de confusion

y_pred = model.predict(X_test)

cm = metrics.confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8,8))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.title('Confusion matrix', size = 15)


<div class="alert alert-block alert-info">
<b>

* La exactitud del modelo es muy alta (aproximadamente 99.99%), lo que indica que el modelo clasifica correctamente la mayoría de las observaciones.

* La especificidad es muy alta (aproximadamente 99.99%), lo que significa que el modelo es muy bueno para identificar correctamente las clases negativas (True Negatives).
Moderada Precisión:

* La precisión es moderada (aproximadamente 87.1%), lo que indica que cuando el modelo predice una clase positiva, es correcto el 87.1% de las veces.

* La sensibilidad es baja (aproximadamente 54.5%), lo que significa que el modelo no es muy bueno para identificar correctamente las clases positivas (True Positives). Esto puede ser un problema si la clase positiva es de particular interés.

* El F1-Score es moderado (aproximadamente 67.1%), lo que refleja un equilibrio entre la precisión y la sensibilidad. Sin embargo, la baja sensibilidad afecta negativamente el F1-Score

</b></div>

### Modificar el desbalance con un peso

Utilizaremos un parámetro adicional en el modelo de Regresión logística en donde indicamos weight = “balanced” y con esto el algoritmo se encargará de equilibrar a la clase minoritaria durante el entrenamiento donde el peso asignado es:

\begin{equation}
w(j) = \frac{n}{K * n(j)}
\end{equation}

donde $n$ es el numero de datos, $K$ es el total de clases y $n(j)$ es el número de datos de cada clase

In [11]:
def run_model_balanced_1(X_train, X_test, Y_train, Y_test):
    clf = LogisticRegression(C=1.0,penalty='l2',random_state=1,solver="newton-cg",class_weight="balanced")
    clf.fit(X_train, Y_train)
    return clf
 

In [ ]:

model_balanced = run_model_balanced_1(X_train, X_test, y_train, y_test)
train_f1, test_f1 = evaluate_model(model_balanced, X_train, y_train, X_test, y_test)



<div class="alert alert-block alert-info">
<b>

El F1-Score es muy bajo (0.12 en entrenamiento y 0.11 en prueba). Esto sugiere que el modelo tiene dificultades para identificar correctamente la clase minoritaria.

Un F1-Score bajo indica que el modelo tiene un equilibrio pobre entre precisión y sensibilidad para la clase minoritaria.

</b></div>

In [ ]:
print(dataset['Class'].value_counts()/len(dataset)*100)

In [14]:
def run_model_balanced_2(X_train, X_test, Y_train, Y_test):
    clf = LogisticRegression(C=1.0,penalty='l2',random_state=1,solver="newton-cg",class_weight={0: 0.2,1: 0.8})
    clf.fit(X_train, Y_train)
    return clf

In [ ]:
model_balanced = run_model_balanced_2(X_train, X_test, y_train, y_test)
train_f1, test_f1 = evaluate_model(model_balanced, X_train, y_train, X_test, y_test)


### NearMiss 

Es un algoritmo que reduce la clase mayoritaria que utiliza un algoritmo similar al k-nearest neighbor para ir seleccionando cuales eliminar.


In [ ]:
us = NearMiss()
X_train_res, y_train_res = us.fit_resample(X_train, y_train)
 
print ("before resampling {}".format(Counter(y_train)))
print ("after resampling {}".format(Counter(y_train_res)))
 
model = run_model(X_train_res, X_test, y_train_res, y_test)
y_pred = model.predict(X_test)
train_f1, test_f1 = evaluate_model(model, X_train_res, y_train_res, X_test, y_test) 

### RandomOverSampler

Crea una muestras nuevas “sintéticas” de la clase minoritaria 

In [ ]:
os =  RandomOverSampler()
X_train_res, y_train_res = os.fit_resample(X_train, y_train)
 
print ("before resampling {}".format(Counter(y_train)))
print ("after resampling {}".format(Counter(y_train_res)))
 
model = run_model(X_train_res, X_test, y_train_res, y_test)
y_pred = model.predict(X_test)
train_f1, test_f1 = evaluate_model(model, X_train_res, y_train_res, X_test, y_test) 


### SMOTE (Synthetic Minority Over-sampling Technique)

SMOTE es un método de sobremuestreo que crea una muestras sintéticas (no duplicadas) de la clase minoritaria, hasta que sea igual a la clase mayoritaria. SMOTE hace esto seleccionando registros similares y alterando ese registro una columna a la vez aleatoriamente.

In [ ]:
os_us = SMOTE()
X_train_res, y_train_res = os_us.fit_resample(X_train, y_train)
 
print ("before resampling {}".format(Counter(y_train)))
print ("after resampling {}".format(Counter(y_train_res)))
 
model = run_model(X_train_res, X_test, y_train_res, y_test)
y_pred = model.predict(X_test)
train_f1, test_f1 = evaluate_model(model, X_train_res, y_train_res, X_test, y_test)

### SMOTETomek

Consiste en aplicar en simultáneo un algoritmo de subsampling y otro de oversampling: SMOTE para oversampling: busca puntos vecinos cercanos y agrega puntos “en linea recta” entre ellos y Tomek para undersampling que quita los de distinta clase que sean vecinos cercanos.

In [ ]:
os_us = SMOTETomek()
X_train_res, y_train_res = os_us.fit_resample(X_train, y_train)
 
print ("before resampling {}".format(Counter(y_train)))
print ("after resampling {}".format(Counter(y_train_res)))
 
model = run_model(X_train_res, X_test, y_train_res, y_test)
y_pred = model.predict(X_test)

train_f1, test_f1 = evaluate_model(model, X_train_res, y_train_res, X_test, y_test)

### Ensamble de Modelos con Balanceo

El ensamble de modelos es una técnica usada para reducir la varianza de las predicciones a través de la combinación de los resultados de varios clasificadores. Además de esto, se aborda el problema de los datos desequilibrados mediante el uso de submuestreo aleatorio para equilibrar la distribución de clases en cada subconjunto. Esto ayuda a reducir el sesgo hacia la clase mayoritaria y mejorar el desempeño de la clase minoritaria.


In [ ]:
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Crear el BalancedBaggingClassifier con el argumento correcto
bbc = BalancedBaggingClassifier(estimator=RandomForestClassifier(),
                                sampling_strategy='auto',
                                replacement=False,
                                random_state=0)


bbc.fit(X_train, y_train)
y_pred = bbc.predict(X_test)
train_f1, test_f1 = evaluate_model(bbc, X_train, y_train, X_test, y_test)
